In [1]:

import os
import time

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

from torchvision import datasets
from torchvision import transforms
import torchvision.models as models

import matplotlib.pyplot as plt
from PIL import Image

#os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [2]:
NUM_CLASSES = 13
BATCH_SIZE = 20
LEARNING_RATE = 0.001
NUM_EPOCHS = 300
DEVICE = 'cuda:1'
device = torch.device('cuda')

In [3]:
data_transform = transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.Grayscale(num_output_channels=3),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
    ])
training_data = datasets.ImageFolder(root='./train',transform=data_transform)
train_loader = torch.utils.data.DataLoader(training_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)

for images, labels in train_loader:  
    print('Image batch dimensions:', images.shape)
    print('Image label dimensions:', labels.shape)
    #print(images)
    #images = images.cuda()
    break



Image batch dimensions: torch.Size([20, 3, 224, 224])
Image label dimensions: torch.Size([20])


In [ ]:
model = models.resnet50(pretrained=True, progress=True)
model.fc = nn.Linear(2048, NUM_CLASSES)
#print(torch.device('cuda'))
#model.to(device)
model = model.cuda()
print(model)
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [4]:
model = models.resnext101_32x8d(pretrained=True, progress=True)
model.fc = nn.Linear(2048, NUM_CLASSES)
print(model)
model = model.cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
      (bn2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1

In [ ]:
model = models.wide_resnet101_2(pretrained=True, progress=False)
model.fc = nn.Linear(2048, NUM_CLASSES)
print(model)
model = model.cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [5]:
#model = torch.load('./resnext101_32x8d')
def compute_accuracy(model, data_loader, device):
    correct_pred, num_examples = 0, 0
    for i, (features, targets) in enumerate(data_loader):
            
        #features = features.to(device)
        #targets = targets.to(device)
        features = features.cuda()
        targets = targets.cuda()
        
        #logits, probas = model(features)
        logits = model(features)
        probas = F.softmax(logits, dim=1)
        
        _, predicted_labels = torch.max(probas, 1)
        num_examples += targets.size(0)
        correct_pred += (predicted_labels == targets).sum()
    return correct_pred.float()/num_examples * 100
def adjust_learning_rate(optimizer, epoch):
    """Sets the learning rate to the initial LR decayed by 10 every 20 epochs"""
    lr = LEARNING_RATE * (0.1 ** (epoch // 20))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr    
best_acc = 0.93
start_time = time.time()
for epoch in range(NUM_EPOCHS):
    
    model.train()
    for batch_idx, (features, targets) in enumerate(train_loader):
        
        #features = features.to(DEVICE)
        #targets = targets.to(DEVICE)
        features = features.cuda()
        targets = targets.cuda()
        
        ### FORWARD AND BACK PROP
        #logits, probas = model(features)
        logits = model(features)
        probas = F.softmax(logits, dim=1)
        #print(logits)
        cost = F.cross_entropy(logits, targets)
        optimizer.zero_grad()
        
        cost.backward()
        
        ### UPDATE MODEL PARAMETERS
        optimizer.step()
        adjust_learning_rate(optimizer, epoch)
        
        ### LOGGING
        if not batch_idx % 30:
            print ('Epoch: %03d/%03d | Batch %04d/%04d | Cost: %.4f' 
                   %(epoch+1, NUM_EPOCHS, batch_idx, 
                     len(train_loader), cost))

        

    model.eval()
    with torch.set_grad_enabled(False): # save memory during inference
        acc = compute_accuracy(model, train_loader, device=DEVICE)
        print('Epoch: %03d/%03d | Train: %.3f%%' % (
              epoch+1, NUM_EPOCHS, acc))
        if acc > best_acc:
            torch.save(model, './resnext101_32x8d_300')
            best_acc = acc
            print("model save at acc = ",acc)
    print('Time elapsed: %.2f min' % ((time.time() - start_time)/60))

print('Total Training Time: %.2f min' % ((time.time() - start_time)/60))

Epoch: 001/300 | Batch 0000/0141 | Cost: 2.6530
Epoch: 001/300 | Batch 0030/0141 | Cost: 1.8372
Epoch: 001/300 | Batch 0060/0141 | Cost: 1.5897
Epoch: 001/300 | Batch 0090/0141 | Cost: 1.1711
Epoch: 001/300 | Batch 0120/0141 | Cost: 1.8129
Epoch: 001/300 | Train: 38.737%
model save at acc =  tensor(38.7371, device='cuda:0')
Time elapsed: 2.84 min
Epoch: 002/300 | Batch 0000/0141 | Cost: 1.3513
Epoch: 002/300 | Batch 0030/0141 | Cost: 1.2992
Epoch: 002/300 | Batch 0060/0141 | Cost: 1.7046
Epoch: 002/300 | Batch 0090/0141 | Cost: 1.3364
Epoch: 002/300 | Batch 0120/0141 | Cost: 1.1968
Epoch: 002/300 | Train: 44.555%
model save at acc =  tensor(44.5548, device='cuda:0')
Time elapsed: 5.66 min
Epoch: 003/300 | Batch 0000/0141 | Cost: 1.0753
Epoch: 003/300 | Batch 0030/0141 | Cost: 0.8736
Epoch: 003/300 | Batch 0060/0141 | Cost: 1.7614
Epoch: 003/300 | Batch 0090/0141 | Cost: 1.3796
Epoch: 003/300 | Batch 0120/0141 | Cost: 1.1022
Epoch: 003/300 | Train: 60.553%
model save at acc =  tensor(60

Epoch: 026/300 | Batch 0000/0141 | Cost: 0.2654
Epoch: 026/300 | Batch 0030/0141 | Cost: 0.1546
Epoch: 026/300 | Batch 0060/0141 | Cost: 0.1638
Epoch: 026/300 | Batch 0090/0141 | Cost: 0.4208
Epoch: 026/300 | Batch 0120/0141 | Cost: 0.4347
Epoch: 026/300 | Train: 88.897%
Time elapsed: 72.88 min
Epoch: 027/300 | Batch 0000/0141 | Cost: 0.1894
Epoch: 027/300 | Batch 0030/0141 | Cost: 0.6471
Epoch: 027/300 | Batch 0060/0141 | Cost: 0.3797
Epoch: 027/300 | Batch 0090/0141 | Cost: 0.2302
Epoch: 027/300 | Batch 0120/0141 | Cost: 0.4714
Epoch: 027/300 | Train: 89.464%
model save at acc =  tensor(89.4644, device='cuda:0')
Time elapsed: 75.68 min
Epoch: 028/300 | Batch 0000/0141 | Cost: 0.6097
Epoch: 028/300 | Batch 0030/0141 | Cost: 0.1157
Epoch: 028/300 | Batch 0060/0141 | Cost: 0.5339
Epoch: 028/300 | Batch 0090/0141 | Cost: 0.3320
Epoch: 028/300 | Batch 0120/0141 | Cost: 0.1630
Epoch: 028/300 | Train: 89.784%
model save at acc =  tensor(89.7836, device='cuda:0')
Time elapsed: 78.48 min
Epoc

Epoch: 052/300 | Batch 0060/0141 | Cost: 0.2679
Epoch: 052/300 | Batch 0090/0141 | Cost: 0.5643
Epoch: 052/300 | Batch 0120/0141 | Cost: 0.2572
Epoch: 052/300 | Train: 91.486%
Time elapsed: 145.65 min
Epoch: 053/300 | Batch 0000/0141 | Cost: 0.3093
Epoch: 053/300 | Batch 0030/0141 | Cost: 0.2928
Epoch: 053/300 | Batch 0060/0141 | Cost: 0.1869
Epoch: 053/300 | Batch 0090/0141 | Cost: 0.3070
Epoch: 053/300 | Batch 0120/0141 | Cost: 0.2363
Epoch: 053/300 | Train: 92.480%
Time elapsed: 148.45 min
Epoch: 054/300 | Batch 0000/0141 | Cost: 0.2833
Epoch: 054/300 | Batch 0030/0141 | Cost: 0.0700
Epoch: 054/300 | Batch 0060/0141 | Cost: 0.3195
Epoch: 054/300 | Batch 0090/0141 | Cost: 0.3181
Epoch: 054/300 | Batch 0120/0141 | Cost: 0.2701
Epoch: 054/300 | Train: 91.841%
Time elapsed: 151.27 min
Epoch: 055/300 | Batch 0000/0141 | Cost: 0.2317
Epoch: 055/300 | Batch 0030/0141 | Cost: 0.2505
Epoch: 055/300 | Batch 0060/0141 | Cost: 0.2297
Epoch: 055/300 | Batch 0090/0141 | Cost: 0.3084
Epoch: 055/30

Epoch: 079/300 | Batch 0000/0141 | Cost: 0.1755
Epoch: 079/300 | Batch 0030/0141 | Cost: 0.2022
Epoch: 079/300 | Batch 0060/0141 | Cost: 0.5507
Epoch: 079/300 | Batch 0090/0141 | Cost: 0.3166
Epoch: 079/300 | Batch 0120/0141 | Cost: 0.1116
Epoch: 079/300 | Train: 92.621%
Time elapsed: 221.38 min
Epoch: 080/300 | Batch 0000/0141 | Cost: 0.1118
Epoch: 080/300 | Batch 0030/0141 | Cost: 0.3351
Epoch: 080/300 | Batch 0060/0141 | Cost: 0.1107
Epoch: 080/300 | Batch 0090/0141 | Cost: 0.3889
Epoch: 080/300 | Batch 0120/0141 | Cost: 0.0920
Epoch: 080/300 | Train: 92.905%
Time elapsed: 224.18 min
Epoch: 081/300 | Batch 0000/0141 | Cost: 0.2424
Epoch: 081/300 | Batch 0030/0141 | Cost: 0.4503
Epoch: 081/300 | Batch 0060/0141 | Cost: 0.1482
Epoch: 081/300 | Batch 0090/0141 | Cost: 0.1213
Epoch: 081/300 | Batch 0120/0141 | Cost: 0.2533
Epoch: 081/300 | Train: 92.870%
Time elapsed: 226.97 min
Epoch: 082/300 | Batch 0000/0141 | Cost: 0.2742
Epoch: 082/300 | Batch 0030/0141 | Cost: 0.1841
Epoch: 082/30

Epoch: 106/300 | Batch 0090/0141 | Cost: 0.6820
Epoch: 106/300 | Batch 0120/0141 | Cost: 0.1958
Epoch: 106/300 | Train: 92.763%
Time elapsed: 296.92 min
Epoch: 107/300 | Batch 0000/0141 | Cost: 0.4082
Epoch: 107/300 | Batch 0030/0141 | Cost: 0.2047
Epoch: 107/300 | Batch 0060/0141 | Cost: 0.1352
Epoch: 107/300 | Batch 0090/0141 | Cost: 0.1957
Epoch: 107/300 | Batch 0120/0141 | Cost: 0.0744
Epoch: 107/300 | Train: 92.231%
Time elapsed: 299.71 min
Epoch: 108/300 | Batch 0000/0141 | Cost: 0.1157
Epoch: 108/300 | Batch 0030/0141 | Cost: 0.4957
Epoch: 108/300 | Batch 0060/0141 | Cost: 0.0693
Epoch: 108/300 | Batch 0090/0141 | Cost: 0.2503
Epoch: 108/300 | Batch 0120/0141 | Cost: 0.1476
Epoch: 108/300 | Train: 92.444%
Time elapsed: 302.50 min
Epoch: 109/300 | Batch 0000/0141 | Cost: 0.3450
Epoch: 109/300 | Batch 0030/0141 | Cost: 0.2542
Epoch: 109/300 | Batch 0060/0141 | Cost: 0.1008
Epoch: 109/300 | Batch 0090/0141 | Cost: 0.3460
Epoch: 109/300 | Batch 0120/0141 | Cost: 0.3326
Epoch: 109/30

...

Epoch: 189/300 | Batch 0030/0141 | Cost: 0.2360
Epoch: 189/300 | Batch 0060/0141 | Cost: 0.0722
Epoch: 189/300 | Batch 0090/0141 | Cost: 0.3853
Epoch: 189/300 | Batch 0120/0141 | Cost: 0.3621
Epoch: 189/300 | Train: 93.260%
Time elapsed: 528.88 min
Epoch: 190/300 | Batch 0000/0141 | Cost: 0.5191
Epoch: 190/300 | Batch 0030/0141 | Cost: 0.3068
Epoch: 190/300 | Batch 0060/0141 | Cost: 0.0514
Epoch: 190/300 | Batch 0090/0141 | Cost: 0.4146
Epoch: 190/300 | Batch 0120/0141 | Cost: 0.5188
Epoch: 190/300 | Train: 92.338%
Time elapsed: 531.68 min
Epoch: 191/300 | Batch 0000/0141 | Cost: 0.1785
Epoch: 191/300 | Batch 0030/0141 | Cost: 0.0924
Epoch: 191/300 | Batch 0060/0141 | Cost: 0.0583
Epoch: 191/300 | Batch 0090/0141 | Cost: 0.2046
Epoch: 191/300 | Batch 0120/0141 | Cost: 0.1342
Epoch: 191/300 | Train: 92.551%
Time elapsed: 534.48 min
Epoch: 192/300 | Batch 0000/0141 | Cost: 0.1368
Epoch: 192/300 | Batch 0030/0141 | Cost: 0.2737
Epoch: 192/300 | Batch 0060/0141 | Cost: 0.3864
Epoch: 192/30

Epoch: 216/300 | Train: 92.196%
Time elapsed: 604.38 min
Epoch: 217/300 | Batch 0000/0141 | Cost: 0.2372
Epoch: 217/300 | Batch 0030/0141 | Cost: 0.7580
Epoch: 217/300 | Batch 0060/0141 | Cost: 0.3989
Epoch: 217/300 | Batch 0090/0141 | Cost: 0.3144
Epoch: 217/300 | Batch 0120/0141 | Cost: 0.2604
Epoch: 217/300 | Train: 92.373%
Time elapsed: 607.17 min
Epoch: 218/300 | Batch 0000/0141 | Cost: 0.2798
Epoch: 218/300 | Batch 0030/0141 | Cost: 0.1761
Epoch: 218/300 | Batch 0060/0141 | Cost: 0.2329
Epoch: 218/300 | Batch 0090/0141 | Cost: 0.4980
Epoch: 218/300 | Batch 0120/0141 | Cost: 0.3402
Epoch: 218/300 | Train: 92.125%
Time elapsed: 609.97 min
Epoch: 219/300 | Batch 0000/0141 | Cost: 0.1978
Epoch: 219/300 | Batch 0030/0141 | Cost: 0.4149
Epoch: 219/300 | Batch 0060/0141 | Cost: 0.1236
Epoch: 219/300 | Batch 0090/0141 | Cost: 0.4072
Epoch: 219/300 | Batch 0120/0141 | Cost: 0.1368
Epoch: 219/300 | Train: 92.692%
Time elapsed: 612.76 min
Epoch: 220/300 | Batch 0000/0141 | Cost: 0.3253
Epoc

Epoch: 244/300 | Batch 0090/0141 | Cost: 0.3505
Epoch: 244/300 | Batch 0120/0141 | Cost: 0.1388
Epoch: 244/300 | Train: 92.551%
Time elapsed: 682.68 min
Epoch: 245/300 | Batch 0000/0141 | Cost: 0.2013
Epoch: 245/300 | Batch 0030/0141 | Cost: 0.2465
Epoch: 245/300 | Batch 0060/0141 | Cost: 0.0744
Epoch: 245/300 | Batch 0090/0141 | Cost: 0.8403
Epoch: 245/300 | Batch 0120/0141 | Cost: 0.4729
Epoch: 245/300 | Train: 92.231%
Time elapsed: 685.47 min
Epoch: 246/300 | Batch 0000/0141 | Cost: 0.4094
Epoch: 246/300 | Batch 0030/0141 | Cost: 0.3833
Epoch: 246/300 | Batch 0060/0141 | Cost: 0.4146
Epoch: 246/300 | Batch 0090/0141 | Cost: 0.2612
Epoch: 246/300 | Batch 0120/0141 | Cost: 0.3564
Epoch: 246/300 | Train: 93.260%
Time elapsed: 688.27 min
Epoch: 247/300 | Batch 0000/0141 | Cost: 0.3206
Epoch: 247/300 | Batch 0030/0141 | Cost: 0.5998
Epoch: 247/300 | Batch 0060/0141 | Cost: 0.0856
Epoch: 247/300 | Batch 0090/0141 | Cost: 0.1737
Epoch: 247/300 | Batch 0120/0141 | Cost: 0.5079
Epoch: 247/30

Epoch: 272/300 | Batch 0030/0141 | Cost: 0.2556
Epoch: 272/300 | Batch 0060/0141 | Cost: 0.4864
Epoch: 272/300 | Batch 0090/0141 | Cost: 0.5817
Epoch: 272/300 | Batch 0120/0141 | Cost: 0.1500
Epoch: 272/300 | Train: 93.331%
Time elapsed: 760.99 min
Epoch: 273/300 | Batch 0000/0141 | Cost: 0.3067
Epoch: 273/300 | Batch 0030/0141 | Cost: 0.3991
Epoch: 273/300 | Batch 0060/0141 | Cost: 0.1217
Epoch: 273/300 | Batch 0090/0141 | Cost: 0.1808
Epoch: 273/300 | Batch 0120/0141 | Cost: 0.3931
Epoch: 273/300 | Train: 92.621%
Time elapsed: 763.79 min
Epoch: 274/300 | Batch 0000/0141 | Cost: 0.6863
Epoch: 274/300 | Batch 0030/0141 | Cost: 0.1259
Epoch: 274/300 | Batch 0060/0141 | Cost: 0.4153
Epoch: 274/300 | Batch 0090/0141 | Cost: 0.3348
Epoch: 274/300 | Batch 0120/0141 | Cost: 0.4823
Epoch: 274/300 | Train: 93.047%
Time elapsed: 766.58 min
Epoch: 275/300 | Batch 0000/0141 | Cost: 0.0933
Epoch: 275/300 | Batch 0030/0141 | Cost: 0.1275
Epoch: 275/300 | Batch 0060/0141 | Cost: 0.1452
Epoch: 275/30

Epoch: 299/300 | Train: 91.699%
Time elapsed: 836.41 min
Epoch: 300/300 | Batch 0000/0141 | Cost: 0.1553
Epoch: 300/300 | Batch 0030/0141 | Cost: 0.5472
Epoch: 300/300 | Batch 0060/0141 | Cost: 0.2865
Epoch: 300/300 | Batch 0090/0141 | Cost: 0.1834
Epoch: 300/300 | Batch 0120/0141 | Cost: 0.2893
Epoch: 300/300 | Train: 92.444%
Time elapsed: 839.20 min
Total Training Time: 839.20 min


In [8]:
import csv
model = torch.load('./resnext101_32x8d_300')
testing_data = datasets.ImageFolder(root='./test1',transform=data_transform)
test_loader = torch.utils.data.DataLoader(testing_data, batch_size=1, shuffle=False, num_workers=4)
label = ['bedroom','coast','forest','highway','insidecity','kitchen','livingroom','mountain','office','opencountry','street','suburb','tallbuilding']
#print(testing_data[1039])
k = 0
with open('output5.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['id','label'])
    for i, (images, labels) in enumerate(test_loader):
        images = images.cuda()
        model.eval()
        logits = model(images)
        probas = F.softmax(logits, dim=1)
        _, predicted_labels = torch.max(probas, 1)
        #print(predicted_labels)
        image_id = 'image_' + "%04d" % k
        #print(image_id)
        print(image_id,label[predicted_labels[0].item()])
        writer.writerow([image_id,label[predicted_labels[0].item()]])
        
        #break
        k+=1

'''
with open('output.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['id','label'])
    for i, (images, labels) in enumerate(test_loader):
        images = images.cuda()
        model.eval()
        logits = model(images)
        probas = F.softmax(logits, dim=1)
        _, predicted_labels = torch.max(probas, 1)

        #print(predicted_labels[0].item())
        image_id = 'image_' + "%04d" % i
        print()
        for j in range(10):
            writer.writerow([image_id,label[]])
        break
''' 


image_0000 highway
image_0001 bedroom
image_0002 suburb
image_0003 bedroom
image_0004 tallbuilding
image_0005 insidecity
image_0006 coast
image_0007 forest
image_0008 coast
image_0009 street
image_0010 highway
image_0011 bedroom
image_0012 mountain
image_0013 street
image_0014 insidecity
image_0015 coast
image_0016 highway
image_0017 livingroom
image_0018 kitchen
image_0019 insidecity
image_0020 highway
image_0021 mountain
image_0022 highway
image_0023 insidecity
image_0024 office
image_0025 highway
image_0026 forest
image_0027 bedroom
image_0028 mountain
image_0029 forest
image_0030 street
image_0031 forest
image_0032 bedroom
image_0033 highway
image_0034 highway
image_0035 suburb
image_0036 tallbuilding
image_0037 coast
image_0038 forest
image_0039 suburb
image_0040 suburb
image_0041 office
image_0042 suburb
image_0043 livingroom
image_0044 livingroom
image_0045 street
image_0046 coast
image_0047 insidecity
image_0048 street
image_0049 tallbuilding
image_0050 kitchen
image_0051 offic

image_0416 office
image_0417 forest
image_0418 livingroom
image_0419 tallbuilding
image_0420 street
image_0421 kitchen
image_0422 coast
image_0423 bedroom
image_0424 coast
image_0425 livingroom
image_0426 street
image_0427 insidecity
image_0428 tallbuilding
image_0429 livingroom
image_0430 forest
image_0431 highway
image_0432 coast
image_0433 opencountry
image_0434 suburb
image_0435 coast
image_0436 office
image_0437 forest
image_0438 street
image_0439 opencountry
image_0440 mountain
image_0441 suburb
image_0442 livingroom
image_0443 street
image_0444 highway
image_0445 office
image_0446 suburb
image_0447 insidecity
image_0448 coast
image_0449 insidecity
image_0450 forest
image_0451 tallbuilding
image_0452 highway
image_0453 mountain
image_0454 suburb
image_0455 office
image_0456 forest
image_0457 suburb
image_0458 office
image_0459 opencountry
image_0460 opencountry
image_0461 coast
image_0462 suburb
image_0463 opencountry
image_0464 office
image_0465 insidecity
image_0466 bedroom
ima

image_0831 livingroom
image_0832 suburb
image_0833 insidecity
image_0834 livingroom
image_0835 opencountry
image_0836 bedroom
image_0837 forest
image_0838 insidecity
image_0839 forest
image_0840 bedroom
image_0841 livingroom
image_0842 bedroom
image_0843 street
image_0844 opencountry
image_0845 tallbuilding
image_0846 tallbuilding
image_0847 bedroom
image_0848 kitchen
image_0849 street
image_0850 highway
image_0851 kitchen
image_0852 opencountry
image_0853 highway
image_0854 office
image_0855 bedroom
image_0856 kitchen
image_0857 livingroom
image_0858 mountain
image_0859 tallbuilding
image_0860 livingroom
image_0861 insidecity
image_0862 highway
image_0863 opencountry
image_0864 suburb
image_0865 bedroom
image_0866 suburb
image_0867 tallbuilding
image_0868 office
image_0869 forest
image_0870 highway
image_0871 bedroom
image_0872 tallbuilding
image_0873 mountain
image_0874 kitchen
image_0875 opencountry
image_0876 highway
image_0877 livingroom
image_0878 forest
image_0879 forest
image_0

'\nwith open(\'output.csv\', \'w\', newline=\'\') as csvfile:\n    writer = csv.writer(csvfile)\n    writer.writerow([\'id\',\'label\'])\n    for i, (images, labels) in enumerate(test_loader):\n        images = images.cuda()\n        model.eval()\n        logits = model(images)\n        probas = F.softmax(logits, dim=1)\n        _, predicted_labels = torch.max(probas, 1)\n\n        #print(predicted_labels[0].item())\n        image_id = \'image_\' + "%04d" % i\n        print()\n        for j in range(10):\n            writer.writerow([image_id,label[]])\n        break\n'

In [7]:
torch.save(model, './resnext101_32x8d_300_last')
#rr = torch.load('./resnet50_first_try')
#rr.eval()
#print(rr)
